<a href="https://colab.research.google.com/github/roman6s/SCM_Fallstudie/blob/main/Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [117]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [118]:
! pip install -q pyscipopt

In [119]:
import pandas as pd
from pyscipopt import Model, quicksum

## Optimierungsmodell

In [120]:
scip = Model()

## Daten laden

In [121]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

In [122]:
# Preisprogonose
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")


## Indexmengen

In [123]:
# Menge der Stunden des Folgetages
T = [str(i) for i in range(1, 25)]

## Parameter

In [124]:
# Nominelle Speicherkapazität in MWh
cap = 40

In [125]:
# Depth of Discharge
DoD = 0.8

In [126]:
# Minimaler Stage of Charge in MWh
SOCmin = 8

In [127]:
# Maximaler Stage of Charge in MWh
SOCmax = 40

In [128]:
# State of Charge
SOC = {t: scip.addVar(vtype="CONTINUOUS", lb=SOCmin, ub=SOCmax, name=f"SOC_{t}") for t in T}

In [129]:
# C-Rate
c = 0.5

In [130]:
# Round Trip Efficiency
eta = 0.95

In [131]:
# Wirkungsgrad
mu = 0.985

In [132]:
# Zyklus
z = {t: scip.addVar(vtype="CONTINUOUS", lb=0, ub=2, name=f"z_{t}") for t in T}

In [133]:
#
P_Markt = preisprognose.groupby("Stunde")["Strompreis"].mean().tolist()

In [134]:
# MinimalPreis Stunde t in €/MWh
Pmin = -500

In [135]:
# Maximalpreis zur Stunde t in €/MWh
Pmax = 4000

In [136]:
# Zykluskosten
cost = 1500

## Entscheidungsvariablen

In [137]:
# Entscheidung x
X={}
for t in T:
  X[t] = scip.addVar(vtype="CONTINUOUS", lb=-(c * cap), ub=c * cap, name=f"X_{t}") # bounds anpassen mit Wirkungsgrad


## Zielfunktion

Max $ZF = \sum_{t \in T} (P_h \cdot x_h) - {\text{cost}} \cdot \text{z}$


In [138]:
# Maximiere
scip.setObjective(
    quicksum(P_Markt[int(t) - 1] * X[t] for t in T) - cost * quicksum(z[t] for t in T),
    sense="maximize"
)

## Nebenbedingungen

**(1) Ladezustandsdynamik**

Formel

$∀ t \in T$

In [139]:
for i, t in enumerate(T):
  if t == "1":
    scip.addCons(SOC[t] == SOCmax * c)
  else:
    previous_t = T[i - 1]  # Get the previous element from T
    scip.addCons(SOC[t] == SOC[previous_t] + eta * mu * X[t] - (1 / (eta * mu)) * X[t])

**(2) Kapazitätsbeschränkung**

Formel

$∀ t \in T$

In [148]:
for t in T:
    # Nebenbedingung 1: Durch Kauf darf nicht mehr geladen werden, als gespeichert werden kann
    scip.addCons(SOC[t] - X[t] <= SOCmax)

    # Nebenbedingung 2: Durch Verkauf darf die SOC nicht unter SOCmin sinken
    scip.addCons(SOC[t] + X[t] >= SOCmin)

**(3) Speicherstand zu Beginn/Ende des Tages**

Formel

$∀ t \in T$

In [149]:
# Nebenbedingung für SOC['1']
scip.addCons(SOC['1'] == 0.5 * cap)

# Nebenbedingung für SOC['24']
scip.addCons(SOC['24'] == 0.5 * cap)

c74

**(4) Zyklenbegrenzung**

Formel

$∀ t \in T$

## Berechnung Lösung

In [ ]:
scip.optimize()
print(scip.getStatus())